<a href="https://colab.research.google.com/github/EngSoftwareEngineer/GNN/blob/main/Parking_Space_Occupancy_LLM_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


#RPTM_reid

In [ ]:
!pip install --upgrade torch

In [ ]:
import os
repo_path = '/content/drive/MyDrive/Colab Notebooks/RPTM_reid-main'  # این مسیر را با مسیر دقیق خودتان جایگزین کنید
os.chdir(repo_path)

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install -r requirements.txt

In [ ]:
!git clone https://github.com/NVIDIA/apex.git
!cd apex


In [ ]:
import os
repo_path = '/content/drive/MyDrive/Colab Notebooks/RPTM_reid-main'  # مسیر درست به پوشه اصلی ریپازیتوری
os.chdir(repo_path)


In [ ]:
!python main.py --config_file configs/veri_r101.yml


In [ ]:
!mkdir logs
!python main.py --config_file configs/veri_r101.yml TEST.WEIGHT '<path to trained model>' TEST.EVAL True

In [ ]:
!pip show apex

#kaggle API

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'engkaffash'
os.environ['KAGGLE_KEY'] = 'b814a26f43bb2071991c9b7a0cb013b8'


#Duck dataset

In [ ]:
import kaggle
import os
import zipfile

# دانلود فایل زیپ از Kaggle
dataset_path = 'whurobin/dukemtmcreid'  # مسیر دیتاست روی Kaggle
output_path = '/content/drive/MyDrive/Colab Notebooks/RPTM_reid-main/data/duck'  # مسیری که می‌خواهید دیتاست را استخراج کنید

# بررسی وجود پوشه مقصد
if not os.path.exists(output_path):
    os.makedirs(output_path)

# دانلود دیتاست
kaggle.api.dataset_download_files(dataset_path, path=output_path, unzip=True)

# استخراج فایل زیپ (اگر unzip=False تنظیم شده باشد)
zip_file_path = os.path.join(output_path, 'dukemtmcreid.zip')
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_path)
    os.remove(zip_file_path)


#Veri Dataset

In [ ]:
import os
import kaggle
import zipfile
import io

# مشخص کردن مسیر برای ذخیره‌سازی داده‌ها
dataset_path = '/content/drive/MyDrive/Colab Notebooks/RPTM_reid-main/data/veri'
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# دانلود فایل از Kaggle
dataset_url = 'abhyudaya12/veri-vehicle-re-identification-dataset'
kaggle.api.dataset_download_files(dataset_url, path=dataset_path, unzip=False)

# نام فایل ZIP دانلود شده
zip_filename = os.path.join(dataset_path, 'veri-vehicle-re-identification-dataset.zip')

# استخراج فایل ZIP
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

print('دانلود و استخراج فایل به پایان رسید.')


#RCNN

In [ ]:
import os
import io
import time
import requests
import zipfile
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# clone the git repository and set it as the working directory
! git clone https://github.com/martin-marek/parking-space-occupancy
os.chdir('parking-space-occupancy')

In [ ]:
# download the dataset
if not os.path.exists('dataset/data'):
    r = requests.get("https://pub-e8bbdcbe8f6243b2a9933704a9b1d8bc.r2.dev/parking%2Frois_gopro.zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('dataset/data')

In [ ]:
from dataset import acpds
from utils import transforms
from utils import visualize as vis

In [ ]:
train_ds, valid_ds, test_ds = acpds.create_datasets('dataset/data')

In [ ]:
image_batch, rois_batch, labels_batch = next(iter(valid_ds))
image_raw, rois, labels = image_batch[0], rois_batch[0], labels_batch[0]
image = transforms.preprocess(image_raw, res=1440)
vis.plot_ds_image(image, rois, labels, show=True)

#اتصال همه به همه

In [ ]:
import networkx as nx

node_ids = list(range(len(rois)))
graph = nx.Graph()

# افزودن نودها به گراف با برچسب‌های پر یا خالی
for i, roi in enumerate(rois):
    label = labels[i].item()
    graph.add_node(node_ids[i], occupied=label)

# اتصال نودها بر اساس مجاورت یا معیارهای دیگر (در اینجا اتصال همه به همه)
for i in range(len(rois)):
    for j in range(i+1, len(rois)):
        graph.add_edge(node_ids[i], node_ids[j])

# نمایش گراف
nx.draw(graph, with_labels=True, node_color=["red" if graph.nodes[i]["occupied"] else "green" for i in graph.nodes])
plt.show()


In [ ]:
!pip install torch_geometric

In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# تبدیل گراف NetworkX به فرمت داده‌ای PyTorch Geometric
data = from_networkx(graph)

# افزودن ویژگی‌های نودها به عنوان یک tensor
data.x = torch.tensor([graph.nodes[i]["occupied"] for i in graph.nodes], dtype=torch.float).unsqueeze(1)

# برچسب‌های نودها (occupied یا vacant) را به data.y اختصاص دهید
# فرض می‌کنیم 1 برای occupied (پر) و 0 برای vacant (خالی)
data.y = torch.tensor([graph.nodes[i]["occupied"] for i in graph.nodes], dtype=torch.long)

# بررسی داده‌ها
print(data)


#GCNConv

In [ ]:
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.optim as optim

# افزودن وزنه‌های کلاس‌ها برای مقابله با عدم تعادل داده‌ها
weights = torch.tensor([0.7, 1.3])  # فرضیه: کلاس 0 (خالی) کمتر است
criterion = nn.CrossEntropyLoss(weight=weights)

# افزایش تعداد لایه‌های GNN یا تغییر نوع مدل
class ImprovedGNN(nn.Module):
    def __init__(self):
        super(ImprovedGNN, self).__init__()
        self.conv1 = GCNConv(1, 32)
        self.conv2 = GCNConv(32, 64)
        self.conv3 = GCNConv(64, 32)
        self.conv4 = GCNConv(32, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = self.conv4(x, edge_index)
        return torch.log_softmax(x, dim=1)

# ادامه مراحل آموزش و ارزیابی با مدل بهبود یافته
model = ImprovedGNN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# آموزش مدل با معماری جدید و وزن‌های کلاس
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum().item()
accuracy = correct / len(data.y)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
from sklearn.metrics import f1_score

model.eval()
with torch.no_grad():
    outputs = model(data)
    _, preds = outputs.max(dim=1)


y_true = data.y.numpy()
y_pred = preds.numpy()

# محاسبه F1-Score
f1 = f1_score(y_true, y_pred, average='weighted')  # 'weighted' برای محاسبه میانگین وزنی
print(f'F1 Score: {f1:.4f}')


#GATConv

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATConv
from torch_geometric.data import Data

# تعریف مدل GNN بهبود یافته
class ImprovedGNN(nn.Module):
    def __init__(self):
        super(ImprovedGNN, self).__init__()
        self.conv1 = GATConv(1, 16)
        self.conv2 = GATConv(16, 32)
        self.conv3 = GATConv(32, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        return torch.log_softmax(x, dim=1)

# تنظیم وزن کلاس‌ها برای تعادل
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]))

# تعریف مدل، optimizer
model = ImprovedGNN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# آموزش مدل
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # نرمال‌سازی گرادیان‌ها
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum().item()
accuracy = correct / len(data.y)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
positions = {i: (np.random.rand(), np.random.rand()) for i in graph.nodes}

# رسم گراف
plt.figure(figsize=(10, 10))
pos = positions
nx.draw(graph, pos, with_labels=True, node_color=["red" if graph.nodes[i]["occupied"] else "green" for i in graph.nodes])
plt.title('Graph Visualization')
plt.show()


In [ ]:
from sklearn.metrics import f1_score

model.eval()
with torch.no_grad():
    outputs = model(data)
    _, preds = outputs.max(dim=1)


y_true = data.y.numpy()
y_pred = preds.numpy()

# محاسبه F1-Score
f1 = f1_score(y_true, y_pred, average='weighted')  # 'weighted' برای محاسبه میانگین وزنی
print(f'F1 Score: {f1:.4f}')


In [ ]:
labels

In [ ]:
pred

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# بارگذاری مدل و توکنایزر
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
llm_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# ساخت متن برای پردازش
text = f"Model GNN Predictions: {pred.tolist()}\nTrue Labels: {data.y.tolist()}\nAccuracy: {accuracy:.4f}\n"

# توکنایز و آماده‌سازی داده‌ها برای مدل LLM
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# پردازش داده‌ها با مدل LLM
with torch.no_grad():
    outputs = llm_model(**inputs)

# خروجی مدل LLM
logits = outputs.logits

# تبدیل logits به احتمالات (اختیاری)
probabilities = torch.softmax(logits, dim=-1)


In [ ]:
from transformers import pipeline

# بارگذاری مدل تولید متن (مثل GPT-2)
text_generator = pipeline('text-generation', model='gpt2')

# تولید توضیحات
generated_text = text_generator(f"Explain the results of GNN model. {text}", max_length=400)
print(generated_text[0]['generated_text'])


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# بارگذاری مدل و توکنایزر
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
llm_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# محاسبه تعداد جاهای خالی و تعداد ماشین‌های پارک شده
num_occupied = (data.y == 1).sum().item()
num_empty = (data.y == 0).sum().item()

# ساخت متن برای پردازش
text = (f"Model GNN Predictions: {pred.tolist()}\n"
        f"True Labels: {data.y.tolist()}\n"
        f"Accuracy: {accuracy:.4f}\n"
        f"Number of parked cars: {num_occupied}\n"
        f"Number of empty spaces: {num_empty}\n")

# توکنایز و آماده‌سازی داده‌ها برای مدل LLM
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# پردازش داده‌ها با مدل LLM
with torch.no_grad():
    outputs = llm_model(**inputs)

# خروجی مدل LLM
logits = outputs.logits

# تبدیل logits به احتمالات (اختیاری)
probabilities = torch.softmax(logits, dim=-1)

# چاپ نتایج
print("Logits:", logits)
print("Probabilities:", probabilities)


In [ ]:
from transformers import pipeline

# بارگذاری مدل تولید متن (مثل GPT-2)
text_generator = pipeline('text-generation', model='gpt2')

# تولید توضیحات
generated_text = text_generator(text, max_new_tokens=150)  # تنظیم تعداد توکن‌های جدید
print(generated_text[0]['generated_text'])


#GrapSAGE

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data

# تعریف مدل GNN بهبود یافته با SAGEConv
class ImprovedGNN_SAGE(nn.Module):
    def __init__(self):
        super(ImprovedGNN_SAGE, self).__init__()
        self.conv1 = SAGEConv(1, 16)  # تغییر از GATConv به SAGEConv
        self.conv2 = SAGEConv(16, 32)  # تغییر از GATConv به SAGEConv
        self.conv3 = SAGEConv(32, 2)   # تغییر از GATConv به SAGEConv

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        return torch.log_softmax(x, dim=1)


In [ ]:
# تنظیم وزن کلاس‌ها برای تعادل
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]))

# تعریف مدل، optimizer
model = ImprovedGNN_SAGE()  # استفاده از مدل SAGEConv
optimizer = optim.Adam(model.parameters(), lr=0.01)

# آموزش مدل
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # نرمال‌سازی گرادیان‌ها
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum().item()
accuracy = correct / len(data.y)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
from sklearn.metrics import f1_score

model.eval()
with torch.no_grad():
    outputs = model(data)
    _, preds = outputs.max(dim=1)


y_true = data.y.numpy()
y_pred = preds.numpy()

# محاسبه F1-Score
f1 = f1_score(y_true, y_pred, average='weighted')  # 'weighted' برای محاسبه میانگین وزنی
print(f'F1 Score: {f1:.4f}')


In [ ]:
labels

In [ ]:
pred

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# بارگذاری مدل و توکنایزر
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
llm_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# ساخت متن برای پردازش
text = f"Model GNN Predictions: {pred.tolist()}\nTrue Labels: {data.y.tolist()}\nAccuracy: {accuracy:.4f}\n"

# توکنایز و آماده‌سازی داده‌ها برای مدل LLM
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# پردازش داده‌ها با مدل LLM
with torch.no_grad():
    outputs = llm_model(**inputs)

# خروجی مدل LLM
logits = outputs.logits

# تبدیل logits به احتمالات (اختیاری)
probabilities = torch.softmax(logits, dim=-1)

In [ ]:
from transformers import pipeline

# بارگذاری مدل تولید متن (مثل GPT-2)
text_generator = pipeline('text-generation', model='gpt2')

# تولید توضیحات
generated_text = text_generator(f"Explain the results of GNN model. {text}", max_length=400)
print(generated_text[0]['generated_text'])


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# بارگذاری مدل و توکنایزر
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
llm_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# محاسبه تعداد جاهای خالی و تعداد ماشین‌های پارک شده
num_occupied = (data.y == 1).sum().item()
num_empty = (data.y == 0).sum().item()

# ساخت متن برای پردازش
text = (f"Model GNN Predictions: {pred.tolist()}\n"
        f"True Labels: {data.y.tolist()}\n"
        f"Accuracy: {accuracy:.4f}\n"
        f"Number of parked cars: {num_occupied}\n"
        f"Number of empty spaces: {num_empty}\n")

# توکنایز و آماده‌سازی داده‌ها برای مدل LLM
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# پردازش داده‌ها با مدل LLM
with torch.no_grad():
    outputs = llm_model(**inputs)

# خروجی مدل LLM
logits = outputs.logits

# تبدیل logits به احتمالات (اختیاری)
probabilities = torch.softmax(logits, dim=-1)

# چاپ نتایج
print("Logits:", logits)
print("Probabilities:", probabilities)

In [ ]:
from transformers import pipeline

# بارگذاری مدل تولید متن (مثل GPT-2)
text_generator = pipeline('text-generation', model='gpt2')

# تولید توضیحات
generated_text = text_generator(text, max_new_tokens=150)  # تنظیم تعداد توکن‌های جدید
print(generated_text[0]['generated_text'])


#GKAN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GKANConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GKANConv, self).__init__(aggr='add')  # یا 'mean', 'max'
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels))
        nn.init.xavier_uniform_(self.att.data)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        return self.propagate(edge_index, x=x)

    def message(self, x_j, edge_index):
        row, col = edge_index
        alpha = torch.matmul(x_j, self.att.squeeze())
        alpha = F.leaky_relu(alpha)
        alpha = F.softmax(alpha, dim=0)
        return alpha.unsqueeze(-1) * x_j

    def update(self, aggr_out):
        return aggr_out

class GKAN(nn.Module):
    def __init__(self):
        super(GKAN, self).__init__()
        self.conv1 = GKANConv(1, 16)
        self.conv2 = GKANConv(16, 32)
        self.conv3 = GKANConv(32, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
import torch.optim as optim
import torch.nn as nn

# تنظیم وزن کلاس‌ها برای تعادل
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]))

# تعریف مدل، optimizer
model = GKAN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# فرض کنید `data` متغیر داده‌ها است
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # نرمال‌سازی گرادیان‌ها
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum().item()
accuracy = correct / len(data.y)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
from sklearn.metrics import f1_score

model.eval()
with torch.no_grad():
    outputs = model(data)
    _, preds = outputs.max(dim=1)


y_true = data.y.numpy()
y_pred = preds.numpy()

# محاسبه F1-Score
f1 = f1_score(y_true, y_pred, average='weighted')  # 'weighted' برای محاسبه میانگین وزنی
print(f'F1 Score: {f1:.4f}')


In [ ]:
labels

In [ ]:
pred

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# بارگذاری مدل و توکنایزر
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
llm_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# ساخت متن برای پردازش
text = f"Model GNN Predictions: {pred.tolist()}\nTrue Labels: {data.y.tolist()}\nAccuracy: {accuracy:.4f}\n"

# توکنایز و آماده‌سازی داده‌ها برای مدل LLM
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# پردازش داده‌ها با مدل LLM
with torch.no_grad():
    outputs = llm_model(**inputs)

# خروجی مدل LLM
logits = outputs.logits

# تبدیل logits به احتمالات (اختیاری)
probabilities = torch.softmax(logits, dim=-1)


In [ ]:
from transformers import pipeline

# بارگذاری مدل تولید متن (مثل GPT-2)
text_generator = pipeline('text-generation', model='gpt2')

# تولید توضیحات
generated_text = text_generator(f"Explain the results of GNN model. {text}", max_length=400)
print(generated_text[0]['generated_text'])


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# بارگذاری مدل و توکنایزر
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
llm_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# محاسبه تعداد جاهای خالی و تعداد ماشین‌های پارک شده
num_occupied = (data.y == 1).sum().item()
num_empty = (data.y == 0).sum().item()

# ساخت متن برای پردازش
text = (f"Model GNN Predictions: {pred.tolist()}\n"
        f"True Labels: {data.y.tolist()}\n"
        f"Accuracy: {accuracy:.4f}\n"
        f"Number of parked cars: {num_occupied}\n"
        f"Number of empty spaces: {num_empty}\n")

# توکنایز و آماده‌سازی داده‌ها برای مدل LLM
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# پردازش داده‌ها با مدل LLM
with torch.no_grad():
    outputs = llm_model(**inputs)

# خروجی مدل LLM
logits = outputs.logits

# تبدیل logits به احتمالات (اختیاری)
probabilities = torch.softmax(logits, dim=-1)

# چاپ نتایج
print("Logits:", logits)
print("Probabilities:", probabilities)



In [ ]:
from transformers import pipeline

# بارگذاری مدل تولید متن (مثل GPT-2)
text_generator = pipeline('text-generation', model='gpt2')

# تولید توضیحات
generated_text = text_generator(text, max_new_tokens=150)  # تنظیم تعداد توکن‌های جدید
print(generated_text[0]['generated_text'])


#اتصال نودهای مجاور در ROI

In [ ]:
node_ids = list(range(len(rois)))
graph = nx.Graph()

# افزودن نودها به گراف با برچسب‌های پر یا خالی
for i, roi in enumerate(rois):
    label = labels[i].item()
    graph.add_node(node_ids[i], occupied=label)

# محاسبه مجاورت و اتصال نودهای مجاور
threshold_distance = 50  # فاصله آستانه برای اتصال نودها (باید تنظیم شود)
for i in range(len(rois)):
    for j in range(i+1, len(rois)):
        # محاسبه فاصله بین مراکز ROIها
        center_i = np.array([(rois[i][0] + rois[i][2]) / 2, (rois[i][1] + rois[i][3]) / 2])
        center_j = np.array([(rois[j][0] + rois[j][2]) / 2, (rois[j][1] + rois[j][3]) / 2])
        dist = np.linalg.norm(center_i - center_j)

        # اگر فاصله کمتر از آستانه بود، نودها را متصل کن
        if dist < threshold_distance:
            graph.add_edge(node_ids[i], node_ids[j])

# نمایش گراف
nx.draw(graph, with_labels=True, node_color=["red" if graph.nodes[i]["occupied"] else "green" for i in graph.nodes])
plt.show()


In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# تبدیل گراف NetworkX به فرمت داده‌ای PyTorch Geometric
data = from_networkx(graph)

# افزودن ویژگی‌های نودها به عنوان یک tensor
data.x = torch.tensor([graph.nodes[i]["occupied"] for i in graph.nodes], dtype=torch.float).unsqueeze(1)

# برچسب‌های نودها (occupied یا vacant) را به data.y اختصاص دهید
# فرض می‌کنیم 1 برای occupied (پر) و 0 برای vacant (خالی)
data.y = torch.tensor([graph.nodes[i]["occupied"] for i in graph.nodes], dtype=torch.long)

# بررسی داده‌ها
print(data)


In [ ]:
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.optim as optim

# افزودن وزنه‌های کلاس‌ها برای مقابله با عدم تعادل داده‌ها
weights = torch.tensor([0.7, 1.3])  # فرضیه: کلاس 0 (خالی) کمتر است
criterion = nn.CrossEntropyLoss(weight=weights)

# افزایش تعداد لایه‌های GNN یا تغییر نوع مدل
class ImprovedGNN(nn.Module):
    def __init__(self):
        super(ImprovedGNN, self).__init__()
        self.conv1 = GCNConv(1, 32)
        self.conv2 = GCNConv(32, 64)
        self.conv3 = GCNConv(64, 32)
        self.conv4 = GCNConv(32, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = self.conv4(x, edge_index)
        return torch.log_softmax(x, dim=1)

# ادامه مراحل آموزش و ارزیابی با مدل بهبود یافته
model = ImprovedGNN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# آموزش مدل با معماری جدید و وزن‌های کلاس
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum().item()
accuracy = correct / len(data.y)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
pred

#GAT

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GATConv

# تبدیل گراف NetworkX به فرمت داده‌ای PyTorch Geometric
data = from_networkx(graph)

# افزودن ویژگی‌های نودها به عنوان یک tensor
data.x = torch.tensor([graph.nodes[i]["occupied"] for i in graph.nodes], dtype=torch.float).unsqueeze(1)

# برچسب‌های نودها (occupied یا vacant) را به data.y اختصاص دهید
data.y = torch.tensor([graph.nodes[i]["occupied"] for i in graph.nodes], dtype=torch.long)

# تعریف مدل GAT
class GAT(nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.conv1 = GATConv(1, 16, heads=4, dropout=0.6)
        self.conv2 = GATConv(16 * 4, 32, heads=4, dropout=0.6)
        self.conv3 = GATConv(32 * 4, 2, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        return torch.log_softmax(x, dim=1)

# ایجاد مدل، تعریف loss و optimizer
model = GAT()
optimizer = optim.Adam(model.parameters(), lr=0.01)
weights = torch.tensor([0.7, 1.3])  # وزنه‌های کلاس‌ها برای مقابله با عدم تعادل
criterion = nn.CrossEntropyLoss(weight=weights)

# آموزش مدل
model.train()
for epoch in range(100):  # تعداد epochها
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# ارزیابی مدل
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum().item()
accuracy = correct / len(data.y)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
pred

In [ ]:
# پیش‌بینی با مدل GNN
preds = test(model, data)

# تبدیل پیش‌بینی‌ها به فرمت متنی
pred_texts = ["Occupied" if pred == 1 else "Free" for pred in preds]

In [ ]:
!pip install openai

In [ ]:
import openai

# تنظیم کلید API برای OpenAI
openai.api_key = '6fd82421b2cf46959a90c98be2b7bfec'

def enhance_predictions_with_llm(pred_texts):
    responses = []
    for text in pred_texts:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # یا مدل LLM دیگر
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Parking labels are {labels} and GNN predictions are {pred_texts}, how many occupied spaces and empty parking spaces do you think?."}
            ],
            max_tokens=50
        )
        responses.append(response.choices[0].message['content'].strip())
    return responses

# بهبود پیش‌بینی‌ها با مدل زبان بزرگ
enhanced_predictions = enhance_predictions_with_llm(pred_texts)


In [ ]:
# چاپ نتایج بهبود یافته
for original, enhanced in zip(pred_texts, enhanced_predictions):
    print(f"Original Prediction: {original}")
    print(f"Enhanced by LLM: {enhanced}")
    print()


#CNN And GNN And LLava

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# بارگذاری مدل ResNet از پیش آموزش دیده
model_cnn = models.resnet50(pretrained=True)
model_cnn.eval()

# حذف لایه‌های آخر برای گرفتن ویژگی‌ها
model_cnn = torch.nn.Sequential(*list(model_cnn.children())[:-1])

# تابع تبدیل ورودی به فرمت قابل استفاده برای مدل ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # تغییر سایز تصویر
    transforms.ToTensor(),          # تبدیل تصویر به tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # نرمال‌سازی
])

def extract_features_from_roi(image_tensor, roi):
    """
    Extracts visual features from a region of interest (ROI) using a pre-trained CNN.

    Args:
    - image_tensor (Tensor): The original image as a Tensor.
    - roi (list or tuple): Region of interest specified as [xmin, ymin, xmax, ymax].

    Returns:
    - feature_vector (Tensor): The extracted features as a tensor.
    """
    # تبدیل Tensor به PIL Image برای برش
    image_pil = transforms.ToPILImage()(image_tensor)

    # اطمینان از اینکه ROI در محدوده تصویر است
    width, height = image_pil.size
    xmin, ymin, xmax, ymax = roi
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    xmax = min(width, xmax)
    ymax = min(height, ymax)

    # استخراج ناحیه مورد نظر از تصویر
    roi_image = image_pil.crop((xmin, ymin, xmax, ymax))

    # اعمال تبدیل‌ها
    roi_tensor = transform(roi_image).unsqueeze(0)  # اضافه کردن بُعد batch

    # استخراج ویژگی‌ها از مدل CNN
    with torch.no_grad():
        feature_vector = model_cnn(roi_tensor)

    return feature_vector.squeeze()  # حذف بُعد اضافی


In [ ]:
import torch
import numpy as np

def create_edges_from_rois(rois, threshold=100):
    """
    Creates edges for a graph based on the ROIs.

    Args:
    - rois (list of lists or tuples): List of ROIs, each specified as [xmin, ymin, xmax, ymax].
    - threshold (float): Maximum distance between ROIs to create an edge.

    Returns:
    - edge_index (Tensor): Tensor of shape [2, num_edges] containing pairs of node indices.
    """
    num_rois = len(rois)
    edge_index = []

    # محاسبه مرکز هر ناحیه (ROI)
    centers = [( (roi[0] + roi[2]) / 2, (roi[1] + roi[3]) / 2 ) for roi in rois]

    # ایجاد یال‌ها بر اساس فاصله مراکز نواحی
    for i in range(num_rois):
        for j in range(i + 1, num_rois):
            distance = np.linalg.norm(np.array(centers[i]) - np.array(centers[j]))
            if distance < threshold:
                edge_index.append([i, j])
                edge_index.append([j, i])  # اضافه کردن یال در هر دو جهت

    # تبدیل به tensor
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    return edge_index


In [ ]:
import torch
import torch_geometric
from torch_geometric.data import Data

# فرض کنید که rois و image_raw در دسترس هستند
# rois شامل نواحی، و image_raw تصویر اصلی است

# 1. استخراج ویژگی‌های هر ناحیه (ROI)
features = []
for roi in rois:
    # استخراج ویژگی‌های بصری از ناحیه با استفاده از یک شبکه CNN
    # اینجا می‌توانید یک مدل CNN از پیش آموزش‌دیده مانند ResNet استفاده کنید
    feature_vector = extract_features_from_roi(image_raw, roi)
    features.append(feature_vector)

# تبدیل ویژگی‌ها به یک tensor
features = torch.tensor(features)

# 2. ایجاد گراف
# ایجاد نودها و یال‌های گراف
# فرض کنید هر نود با نودهای نزدیک خود ارتباط دارد
edge_index = create_edges_from_rois(rois)

# ایجاد گراف داده‌ها
data = Data(x=features, edge_index=edge_index)

# 3. ورودی دادن گراف به GNN
# فرض کنید gnn_model مدل GNN شماست
gnn_model = MyGNNModel()  # این باید از یک کلاس از پیش تعریف شده باشد

# پردازش گراف توسط GNN
out = gnn_model(data)

# 4. تفسیر خروجی
# فرض کنید که خروجی یک پیش‌بینی binary برای هر نود است
predictions = (out > 0.5).float()

# نمایش نتایج
for i, pred in enumerate(predictions):
    if pred == 1:
        print(f"ROI {i} (region: {rois[i]}) is occupied.")
    else:
        print(f"ROI {i} (region: {rois[i]}) is empty.")


#CLIP

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# بارگذاری مدل CLIP و پردازشگر آن
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# تبدیل تصویر پیش‌پردازش شده به PIL image برای ورودی CLIP
image_pil = Image.fromarray((image.permute(1, 2, 0).numpy() * 255).astype(np.uint8))

# پردازش تصویر با استفاده از CLIPProcessor
inputs = processor(images=image_pil, return_tensors="pt")

# استخراج ویژگی‌ها از مدل CLIP
with torch.no_grad():
    features = model.get_image_features(**inputs)

n = len(rois_batch[0])  # تعداد ROIها در هر تصویر
features = torch.randn((n, 512))  # فرضی که هر ROI دارای ویژگی‌های [512] است

# نمایش ویژگی‌های استخراج‌شده
print(features.shape)  # بررسی ابعاد ویژگی‌ها


In [ ]:
!pip install torch_geometric

In [ ]:
from itertools import combinations
import torch
from torch_geometric.data import Data

# فرض کنید که features شامل ویژگی‌های استخراج‌شده از هر فضای پارکینگ است
x = features  # ویژگی‌های نودها

edges = list(combinations(range(n), 2))
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# برای گراف غیرجهت‌دار، یال‌ها را دوباره به ترتیب معکوس اضافه می‌کنیم
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

# ساختار داده‌ی گراف
data = Data(x=x, edge_index=edge_index, y=labels)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# پارامترهای مدل
in_channels = features.shape[1]  # تعداد ویژگی‌های ورودی (ابعاد ویژگی‌های تصویری)
hidden_channels = 16  # تعداد نودهای مخفی
out_channels = 2  # خروجی‌های نهایی (دو کلاس: اشغال یا خالی بودن)

# ایجاد مدل GCN
model = GCN(in_channels, hidden_channels, out_channels)

In [ ]:
import torch.optim as optim

# تعریف optimizer و loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
model.train()
for epoch in range(200):  # 200 دوره برای آموزش
    optimizer.zero_grad()  # صفر کردن گرادیان‌ها
    out = model(data)  # عبور داده از مدل GCN
    loss = criterion(out, labels)  # محاسبه loss
    loss.backward()  # محاسبه گرادیان‌ها
    optimizer.step()  # به‌روزرسانی وزن‌ها

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)  # پیش‌بینی کلاس‌ها
    correct = (pred == labels).sum().item()  # تعداد پیش‌بینی‌های صحیح
    accuracy = correct / len(labels)  # محاسبه دقت
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
!pip install transformers torch


In [ ]:
from transformers import BertTokenizer, BertModel

# بارگذاری مدل BERT و tokenizer آن
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# نمونه متن
texts = [
    "The parking space is occupied by a Blue car.",
    "The parking space is empty.",
    "The parking space is occupied by a Red car.",
    "The parking space is occupied by a Yellow car.",
    "The parking space is occupied by a Green car.",
    "The parking space is occupied by a Black car.",
    "The parking space is occupied by a White car.",
    "The parking space is occupied by a blue car.",
    "The parking space is occupied by a Orange car.",
    "The parking space is occupied by a Gold car.",
    "The parking space is occupied by a Pink car.",
    "The parking space is occupied by a Purple car.",
    "The parking space is occupied by a magenta car.",
    "The parking space is occupied by a cyan car.",
    "The parking space is occupied by a Brown car.",
    "The parking space is occupied by a Gray car.",
    "The parking space is occupied by a Turquoise car.",
    "The parking space is occupied by a Light Blue car.",
    "The parking space is occupied by a Light Green car.",
    "The parking space is occupied by a Pale Yellow car.",
    "The parking space is occupied by a Dark Orange car.",
    "The parking space is occupied by a Dark Blue car.",
    "The parking space is occupied by a Dark Green car.",
    "The parking space is occupied by a Dark Red car.",
    "The parking space is occupied by a Dark Purple car.",
    "The parking space is occupied by a Light Pink car.",
    "The parking space is occupied by a Light Brown car.",
    "The parking space is occupied by a Silver car.",
    "The parking space is occupied by a Bronze car.",
    "The parking space is occupied by a Teal car.",
    "The parking space is occupied by a Azure car.",
    "The parking space is occupied by a Sky Blue car.",
    "The parking space is occupied by a Magenta car.",
    "The parking space is occupied by a Olive Green car.",
    "The parking space is occupied by a Beige car.",
    "The parking space is occupied by a Fuchsia car.",
    "The parking space is occupied by a Emerald Green car.",
    "The parking space is occupied by a Indigo car.",
    "The parking space is occupied by a Khaki car.",
    "The parking space is occupied by a Maroon car.",
    "The parking space is occupied by a Brick Red car.",
    "The parking space is occupied by a Ruby Red car.",
    "The parking space is occupied by a Olive car.",
    "The parking space is occupied by a Navy car.",
    "The parking space is occupied by a Crimson car.",
    "The parking space is occupied by a Wheat car.",
    "The parking space is occupied by a Cobalt Blue car.",
    "The parking space is occupied by a Champagne car.",
    "The parking space is occupied by a Lemon car.",
    "The parking space is occupied by a Eggplant car.",
    "The parking space is occupied by a Ice Blue car.",
    "The parking space is occupied by a Cream car.",
    "The parking space is occupied by a Coral car.",
    "The parking space is occupied by a Navy Blue car.",
    "The parking space is occupied by a Golden Yellow car.",
    "The parking space is occupied by a Mint Green car.",
    "The parking space is occupied by a Dark Brown car.",
    "The parking space is occupied by a Deep Magenta car.",
    "The parking space is occupied by a Silver Gray car.",
    "The parking space is occupied by a Brick Orange car.",
    "The parking space is occupied by a Banana Yellow car.",
    "The parking space is occupied by a Wine Red car.",
    "The parking space is occupied by a Pumpkin Orange car.",
    "The parking space is occupied by a Ocean Blue car.",
    "The parking space is occupied by a Rose Pink car.",
    "The parking space is occupied by a Copper car.",
    "The parking space is occupied by a Light Purple car.",
    "The parking space is occupied by a Herbal Green car.",
    "The parking space is occupied by a Peach car."
]

# تبدیل متن‌ها به token و استخراج ویژگی‌ها
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)
    text_features = outputs.last_hidden_state.mean(dim=1)  # میانگین‌گیری روی طول توکن‌ها

print(text_features.shape)  # ابعاد ویژگی‌های متنی


In [ ]:
# فرض کنید ویژگی‌های تصویری را از قبل داریم
combined_features = torch.cat([features, text_features], dim=1)

# جایگزینی ویژگی‌های نودها با ویژگی‌های ترکیبی
data.x = combined_features


In [ ]:
# تنظیم مدل برای آموزش دوباره
model = GCN(combined_features.shape[1], hidden_channels, out_channels)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# آموزش مدل GNN با ویژگی‌های ترکیبی
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# ارزیابی مدل
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred == labels).sum().item()
    accuracy = correct / len(labels)

print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# مدل در حالت ارزیابی قرار داده می‌شود
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)  # پیش‌بینی کلاس‌ها
    true_labels = labels

# محاسبه معیارهای مختلف
accuracy = accuracy_score(true_labels, pred)
precision = precision_score(true_labels, pred, average='macro')
recall = recall_score(true_labels, pred, average='macro')
f1 = f1_score(true_labels, pred, average='macro')
cm = confusion_matrix(true_labels, pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'Confusion Matrix:\n{cm}')


In [ ]:
# به عنوان مثال، کاهش نرخ یادگیری
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# اضافه کردن Dropout به مدل GNN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# بازآموزی مدل با Dropout
model = GCN(combined_features.shape[1], hidden_channels, out_channels)


In [ ]:
from torch_geometric.data import DataLoader

# ایجاد چندین batch برای cross-validation
data_loader = DataLoader([data], batch_size=1, shuffle=True)

# اجرای cross-validation
for fold, batch in enumerate(data_loader):
    model.train()
    optimizer.zero_grad()
    out = model(batch)
    loss = criterion(out, batch.y)
    loss.backward()
    optimizer.step()
    print(f'Fold {fold}, Loss: {loss.item()}')


In [ ]:
# ذخیره مدل نهایی
torch.save(model.state_dict(), 'final_gnn_model.pth')

# بارگذاری مدل برای استفاده در آینده
model.load_state_dict(torch.load('final_gnn_model.pth'))
model.eval()


In [ ]:
model.eval()  # مدل را در حالت ارزیابی قرار می‌دهیم
with torch.no_grad():
    # داده‌ها باید به شکل مناسب برای مدل ارسال شوند
    out = model(data)
    predictions = out.argmax(dim=1)  # پیش‌بینی برچسب‌ها برای هر نود

In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)  # پیش‌بینی کلاس‌ها
    empty_spaces = (pred == labels).sum().item()  # تعداد پیش‌بینی‌های صحیح
    total_spaces = len(labels)
    filled_spaces = total_spaces - empty_spaces

print(f'Number of empty spaces: {empty_spaces}')
print(f'Number of filled spaces: {filled_spaces}')

#llava-1.5-7b-hf

In [ ]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
from transformers import pipeline

model_id = "llava-hf/llava-1.5-7b-hf"

pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

In [ ]:
max_new_tokens = 200
prompt = "USER: <image>\nHow many empty and full spaces are there in this parking lot?\nASSISTANT:"

outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})

In [ ]:
print(outputs[0]["generated_text"])

#LLAVA

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
!cd LLaVA

In [ ]:
!conda create -n llava python=3.10 -y
!conda activate llava
!python -m pip install --upgrade pip  # enable PEP 660 support
!pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu117
!pip install -e .
!pip uninstall bitsandbytes


In [ ]:
from transformers import CLIPModel, CLIPProcessor

# دانلود مدل و پردازشگر CLIP از Hugging Face
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
from PIL import Image
import torch

# بارگذاری تصویر
image = Image.open("/content/parking-space-occupancy/dataset/data/images/GOPR0024.JPG")

# پردازش تصویر با استفاده از CLIPProcessor
inputs = processor(images=image, return_tensors="pt")

# عبور دادن تصویر از طریق مدل
with torch.no_grad():
    outputs = model.get_image_features(**inputs)

# نمایش ویژگی‌های تصویر
print(outputs)


#CNN+llava-1.5-7b-hf

In [ ]:
!pip install torch torchvision transformers

In [ ]:
!pip install transformers
!pip install git+https://github.com/liuhaotian/LLaVA.git


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# استفاده از مدل BLIP
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

model.to(device)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

# تعریف CNN بهبود یافته
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)  # دو کلاس: پارک شده / خالی

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# لود داده‌های آموزشی
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root='/content/parking-space-occupancy/dataset', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# تعریف مدل، تابع هزینه و بهینه‌ساز
cnn_model = ImprovedCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# آموزش مدل CNN
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for epoch in range(10):  # تعداد epoch ها را می‌توانید تغییر دهید
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

train(cnn_model, train_loader, criterion, optimizer, device)


In [ ]:
from PIL import Image

# تابع پیش‌بینی نهایی
def predict_parking_occupancy(image_path):
    # پیش‌پردازش تصویر
    image = Image.open(image_path)
    input_image = transform(image).unsqueeze(0).to(device)

    # پیش‌بینی با مدل CNN
    cnn_model.eval()
    with torch.no_grad():
        cnn_output = cnn_model(input_image)
        _, predicted = torch.max(cnn_output, 1)

    # استفاده از مدل Llava-1.5-7B-HF برای تحلیل تصویر
    inputs = processor(image, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    text_output = processor.decode(outputs[0], skip_special_tokens=True)

    # ترکیب نتایج CNN و Llava
    if predicted.item() == 1:  # فرض بر این که 1 به معنای پارک شده است
        return f"Parking Occupied. Llava says: {text_output}"
    else:
        return f"Parking Free. Llava says: {text_output}"

# نمونه استفاده
print(predict_parking_occupancy('/content/parking-space-occupancy/dataset/data/images/GOPR6590.JPG'))


#faster_RCNN

In [ ]:
import cv2
import numpy as np
import torchvision.transforms as T
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from google.colab.patches import cv2_imshow  # برای نمایش تصاویر در Colab

# بارگیری مدل Faster R-CNN از پیش‌آموزش‌دیده شده
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# بارگیری تصویر و تبدیل آن به فرمت قابل استفاده برای مدل
def transform_image(image):
    transform = T.Compose([T.ToTensor()])
    return transform(image).unsqueeze(0)  # اضافه کردن ابعاد batch

# تشخیص اشیاء در تصویر
def detect_objects(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    tensor_image = transform_image(image_rgb)

    with torch.no_grad():
        predictions = model(tensor_image)

    return image, predictions

# رسم جعبه‌های محدودکننده
def draw_boxes(image, predictions):
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    threshold = 0.5  # آستانه برای نمایش جعبه‌های محدودکننده

    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i].astype(int)
            x1, y1, x2, y2 = box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return image

# مسیر تصویر ورودی
image_path = '/content/drive/MyDrive/Colab Notebooks/parking-space-occupancy-main/dataset/data/images/GOPR6590.JPG'

# تشخیص اشیاء و رسم جعبه‌های محدودکننده
image, predictions = detect_objects(image_path)
image_with_boxes = draw_boxes(image, predictions)

# نمایش تصویر در Google Colab
cv2_imshow(image_with_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()


#ُStep 1

In [ ]:
import cv2
import numpy as np
import torchvision.transforms as T
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from google.colab.patches import cv2_imshow  # برای نمایش تصاویر در Google Colab

# بارگیری مدل Faster R-CNN از پیش‌آموزش‌دیده شده
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# بارگیری تصویر و تبدیل آن به فرمت قابل استفاده برای مدل
def transform_image(image):
    transform = T.Compose([T.ToTensor()])
    return transform(image).unsqueeze(0)  # اضافه کردن ابعاد batch

# تشخیص اشیاء در تصویر
def detect_objects(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    tensor_image = transform_image(image_rgb)

    with torch.no_grad():
        predictions = model(tensor_image)

    return image, predictions

# رسم جعبه‌های محدودکننده و اختصاص Local ID
def draw_boxes_with_ids(image, predictions):
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    threshold = 0.5  # آستانه برای نمایش جعبه‌های محدودکننده

    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i].astype(int)
            x1, y1, x2, y2 = box

            # رسم جعبه محدودکننده
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # رسم Local ID
            local_id = i + 1  # اختصاص یک ID یکتا به هر ماشین (شروع از 1)
            label = f'ID: {local_id}'
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return image

# مسیر تصویر ورودی
image_path = '/content/drive/MyDrive/Colab Notebooks/parking-space-occupancy-main/dataset/data/images/GOPR6590.JPG'

# تشخیص اشیاء و رسم جعبه‌های محدودکننده با Local ID
image, predictions = detect_objects(image_path)
image_with_boxes_and_ids = draw_boxes_with_ids(image, predictions)

# نمایش تصویر در Google Colab
cv2_imshow(image_with_boxes_and_ids)
cv2.waitKey(0)
cv2.destroyAllWindows()


#Step 2

In [ ]:
import cv2
import numpy as np
import torch
import networkx as nx  # برای ساخت گراف
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from google.colab.patches import cv2_imshow
import torchvision.transforms as T

# بارگیری مدل Faster R-CNN از پیش‌آموزش‌دیده شده
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# بارگیری تصویر و تبدیل آن به فرمت قابل استفاده برای مدل
def transform_image(image):
    transform = T.Compose([T.ToTensor()])
    return transform(image).unsqueeze(0)  # اضافه کردن ابعاد batch

# تشخیص اشیاء در تصویر
def detect_objects(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    tensor_image = transform_image(image_rgb)

    with torch.no_grad():
        predictions = model(tensor_image)

    return image, predictions

# رسم جعبه‌های محدودکننده و اختصاص Local ID
def draw_boxes_with_ids(image, predictions):
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    threshold = 0.5  # آستانه برای نمایش جعبه‌های محدودکننده
    local_ids = []

    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i].astype(int)
            x1, y1, x2, y2 = box

            # رسم جعبه محدودکننده
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # رسم Local ID
            local_id = i + 1  # اختصاص یک ID یکتا به هر ماشین (شروع از 1)
            label = f'ID: {local_id}'
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # ذخیره Local ID و مرکز جعبه محدودکننده
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            local_ids.append((local_id, center_x, center_y))

    return image, local_ids

# ایجاد گراف از Local ID ها
def create_graph(local_ids, distance_threshold=100):
    G = nx.Graph()

    # افزودن نودها
    for local_id, x, y in local_ids:
        G.add_node(local_id, pos=(x, y))

    # افزودن یال‌ها بر اساس فاصله
    for i in range(len(local_ids)):
        for j in range(i + 1, len(local_ids)):
            id1, x1, y1 = local_ids[i]
            id2, x2, y2 = local_ids[j]
            distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            if distance < distance_threshold:
                G.add_edge(id1, id2)

    return G

# مسیر تصویر ورودی
image_path = '/content/drive/MyDrive/Colab Notebooks/parking-space-occupancy-main/dataset/data/images/GOPR6590.JPG'

# تشخیص اشیاء و رسم جعبه‌های محدودکننده با Local ID
image, predictions = detect_objects(image_path)
image_with_boxes_and_ids, local_ids = draw_boxes_with_ids(image, predictions)

# ایجاد گراف
G = create_graph(local_ids)

# نمایش گراف
pos = nx.get_node_attributes(G, 'pos')
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, edge_color='gray')

# نمایش تصویر در Google Colab
cv2_imshow(image_with_boxes_and_ids)
cv2.waitKey(0)
cv2.destroyAllWindows()


#ُStep 3

In [ ]:
!pip install torch_geometric

In [ ]:
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# تبدیل گراف به فرمت قابل استفاده برای GNN
def convert_graph_to_gnn_data(G):
    nodes = list(G.nodes())
    edges = list(G.edges())

    # ساخت لیست از ویژگی‌های نودها (در اینجا فقط موقعیت نودها را در نظر می‌گیریم)
    x = torch.tensor([G.nodes[node]['pos'] for node in nodes], dtype=torch.float)

    # ساخت لیست از یال‌ها
    edge_index = torch.tensor([[nodes.index(edge[0]), nodes.index(edge[1])] for edge in edges], dtype=torch.long).t().contiguous()

    # برچسب‌ها (در اینجا فرض شده که همه نودها باید به یک کلاس مشخص تعلق داشته باشند؛ نیاز به برچسب‌گذاری دارید)
    y = torch.tensor([0] * len(nodes), dtype=torch.long)  # جایگذاری y با برچسب‌های واقعی

    data = Data(x=x, edge_index=edge_index, y=y)
    return data

# تبدیل گراف به داده GNN
gnn_data = convert_graph_to_gnn_data(G)


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return x


In [ ]:
# پارامترهای مدل
num_node_features = gnn_data.num_node_features
hidden_channels = 16
num_classes = 2  # فرض کنیم دو کلاس داشته باشیم: اشغال شده و خالی

# ساخت مدل
model = GCN(num_node_features, hidden_channels, num_classes)

# تعریف optimizer و loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# آموزش مدل
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(gnn_data)
    loss = criterion(out, gnn_data.y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


In [ ]:
model.eval()
with torch.no_grad():
    out = model(gnn_data)
    pred = out.argmax(dim=1)
    print('Predictions:', pred)
